# Fonction renvoyant une durée restée dans un lieu selon l'heure et le lieu

In [117]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import ast
from scipy.stats import invgamma

!pip install openpyxl

In [118]:
#Lire le tableau des lois de durée restée dans un lieu:
tableau_duree= pd.read_excel("data/lois_duree.xlsx")

In [119]:
tableau_duree=tableau_duree.set_index("Unnamed: 0").rename_axis("Plage horaire")

In [120]:
tableau_duree=tableau_duree.map(lambda x: ast.literal_eval(x))


In [121]:
tableau_duree.head()

,Parking,Domicile,Sans,Rue,Entreprise
Plage horaire,,,,,
Matin,"[invgamma, (1.390737571634801, -9.772127964194...","[beta, (2.1872413962120696, -0.005988001327689...","[beta, (1.218287798623423, -5.139663880520553e...","[invgauss, (1.3395802550303704, -1.79049075497...","[loggamma, (200.9195815067319, -27.94843812163..."
Midi,"[pareto, (1.4013317752688952, -1.0965230799343...","[beta, (1.450000395484929, -2.2827792178756118...","[beta, (1.2651197309118647, -1.993472379916114...","[chi2, (1.3400808769713695, -5.127648667537803...","[invgamma, (23.59867259214333, -4.103398702392..."
Après-midi,"[invgamma, (1.512925644318115, -1.407890880051...","[loggamma, (1.9381474601884947, -3.29351895990...","[beta, (1.2182841162030988, -2.040599168036255...","[chi2, (1.3380657374339826, -2.022860775150514...","[gamma, (1.8472666355219776, -3.09093814367362..."
Soir,"[invgauss, (1.5202481297360722, -1.14193992426...","[loggamma, (1.6666258684844566, -4.05430486781...","[beta, (1.4241375370812803, -9.386663096812749...","[beta, (1.6969836415054367, -1.747498811137531...","[beta, (5.436846358958293, -0.3628406844398816..."


In [128]:
def duree_lieu(heure_arrivee,lieu):
    if heure_arrivee<=11:
        plage= "Matin"
    elif heure_arrivee<=14:
        plage="Midi"
    elif heure_arrivee<=17:
        plage= "Après-midi"
    else:
        plage= "Soir"
    loi=tableau_duree.loc[plage,lieu]
    print(loi[0])
    sample=0
    if loi[0] == 'norm':
        # Paramètres de la distribution normale (moyenne, écart-type)
        # Générer un seul nombre suivant la distribution normale
        sample = np.random.normal(loc=round(loi[1][0],4), scale=round(loi[1][1],4))
    elif loi[0]  == 'beta':
        # Paramètres de la distribution bêta (a, b)
        # Générer un seul nombre suivant la distribution bêta
        sample = np.random.beta(a=round(loi[1][0],4), b=round(loi[1][-1],4))
    elif loi[0]  == 'gamma':
        # Paramètres de la distribution gamma (a, scale)
        # Générer un seul nombre suivant la distribution gamma
        sample = np.random.gamma(a=round(loi[1][0],4),loc=round(loi[1][1]), scale=round(loi[1][2],4))
    elif loi[0] == 'pareto':
        # Paramètres de la distribution de Pareto (b, loc, scale)
        sample = np.random.pareto(a=loi[1][0], scale=loi[1][2])
    elif loi[0] == 't':           
        # Paramètres de la distribution t de Student (df, loc, scale)
        sample = np.random.standard_t(df=loi[1][0], loc=loi[1][1], scale=loi[1][2])
    elif loi[0] == 'lognorm':
        # Paramètres de la distribution log-normale (s, loc, scale)
        sample = np.random.lognormal(mean=loi[1][1], sigma=loi[1][0])
    elif loi[0] == 'invgamma':
        # Paramètres de la distribution inverse-gamma (a, loc, scale)
         sample = invgamma.rvs(a=loi[1][0], scale=loi[1][-1])
    elif loi[0] == 'loggamma':
        # Paramètres de la distribution log-gamma (c, loc, scale)
        sample = np.random.loggamma(c=loi[1][0], scale=loi[1][1])
    elif loi[0] == 'invgauss':
         # Paramètres de la distribution inverse-Gaussienne (mu, loc, scale)
        sample = np.random.invgauss(mu=loi[1][0], loc=loi[1][1], scale=loi[1][2])
    elif loi[0] == 'chi2':
        # Paramètres de la distribution de chi carré (df, loc, scale)
        sample = np.random.chisquare(df=loi[1][0])
    return sample

In [ ]:
duree_lieu(19,"Domicile")

In [125]:
tableau_duree.loc["Après-midi","Rue"]

['chi2', (1.3380657374339826, -2.022860775150514e-28, 1.8494210080453803)]